## The data grabbed from UHF antenna

The Parallella (Pcko) grabbing unit was used to capture triggered data from loop antennas.

In [ ]:
# Tato bunka slouzi pro papermill, je oznacena tagem 'parameters'

path = "/storage/experiments/2022/0806_bourka_zdiby/parallella-VLF/"
path = "/storage/experiments/2022/0815_bourka_SlanyMelnik_CAR2/parallella/"
path = "/storage/experiments/2022/0815_bourka_kralupy_CAR0/parallella-VLF/"
path = "/storage/experiments/2022/0815_bourka_benesov_CAR1/parallella/"
path = "/storage/experiments/2022/0816_test_STP_gain/"
path = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF"
path = "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/"
path = "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/"
path = "/storage/experiments/2022/0826_bourka_staryvestec_CAR2/parallella-VLF/"
path = "/storage/experiments/2022/0826_bourka_staryvestec_CAR0/parallella-VLF/"
path = "/storage/experiments/2022/0826_bourka_staryvestec_CAR1/parallella/"
path = "/storage/experiments/2022/0827_bourka_morava_CAR0/parallella-VLF/"
#path = "/storage/experiments/2022/0827_bourka_morava_CAR0/parallella-VLF"
#path = "/storage/experiments/2022/0805_bourka_kourim/parallella-VLF/"
#path = "/storage/experiments/2022/0714_bourka_horovice_CAR1/parallella/"

In [ ]:
import os
import sys
#sys.path.append('..')
#from QFH_lightning.readrec import *
from readrec import *
import matplotlib
from scipy.signal import find_peaks
import datetime


sps = 10e6 # 10 Msps
files = os.listdir(path)
times = []

for fn in files:
    with open(os.path.join(path, fn), 'rb') as f:
        if "png" not in fn and "csv" not in fn:
            try:
                trigger_time = datetime.datetime.strptime(fn[4:]+"0000", "%y%m%d-%H%M%S.%f")
                times.append([path+fn, str(trigger_time)])
                #h, samples, synclog = loadrec(f, skipsamples=True)
            except Exception as e:
                print(fn, e)
                continue

print("Budou se pouzivat soubory", files)
print("Ze slozky", path)


with open(path+'/../triggers.csv', 'w') as f:
    for t in times:
        f.writelines(";".join(t)+";\n")

## Plotting of the data

In [ ]:
import os
pre_trigger_blocks = 380
post_trigger_blocks = 380


pre_trigger_blocks = 100
post_trigger_blocks = 100


matplotlib.rcParams.update({'font.size':15})

for fn in reversed(files):
    print(fn)
    if ".csv" in files:
        continue
    with open(os.path.join(path, fn), 'rb') as f:
        try:
            print("File:", fn)
            h, samples, synclog = loadrec(f)
            t2s, s2t, ticker, formatter = assign_time_axis(fn, h, synclog)
            #print("SYNCLOG", synclog)
            for s in synclog:
                print(s[1], s[0], s2t(s[0]))
            #print(edges((h, samples, synclog)) )
            fig = selective_plotrec(h, samples, synclog, os.path.basename(fn),
                    title=os.path.basename(fn),
                    pre_trigger_blocks = pre_trigger_blocks,
                    post_trigger_blocks = post_trigger_blocks,
                    marktimes=ticker,
                    channels = [0,1,3],
                    #channels = [0,1,2,3,4,5,6,7]
                    )
            
            plt.tight_layout()
            try:
                plt.savefig(os.path.join(path, fn)+'.png')
                pass
            except Exception as e:
                print(e)
            #plt.savefig('/home/roman/VLF.png')
            plt.show()
   
        except Exception as e:
            print(e)

In [ ]:
a = pd.DataFrame(synclog).sort_values(["f", "w"]).reset_index()
a["w"] = a["w"].diff()
a

Plot all lightning records (High memory and time consuming)